# Análisis de Datos
## TP N 1
----
### Grupo N° 7
- Aviani, José
- Díaz, José Luis
- Silvera, Ricardo

---
## Introducción

Para este trabajo elegimos el el dataset Precios Claros – Base SEPA, perteneciente al “Sistema Electrónico de Publicidad de Precios Argentinos (SEPA)" (https://datos.gob.ar/), el cual reúne los precios de comercios minoristas (grandes establecimientos) de más de 70 mil productos en toda la Argentina. Particularmente para este trabajo, seleccionamos el set de datos del establecimiento **Carrefour** ya que era el de mayor tamaño, lo cual es deseable como entrada en un problema de aprendizaje de máquina.
A continuación realizamos el análisis exploratorio de los datos y finalizamos con las conclusiones obtenidas del trabajo.


### Importación de librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import missingno as msno
import scipy.stats as ss

## Análisis Exploratorio de los Datos

### Carga y comprensión de los datos


Los datos están separados en 3 archivos
- comercio
- sucursales
- productos

Cada uno contiene información específica sobre la entidad que indica su nombre. Vamos a investigar a cada uno para comprender la información que contienen y como está organizada y unificar todos los datos que nos interesen en un único dataframe. 

In [ ]:
#Comercio
comercio=pd.read_csv('./dataset/comercio.csv',delimiter='|')
#Sucursales
sucursales=pd.read_csv('./dataset/sucursales.csv',delimiter='|')
#Productos
productos=pd.read_csv('./dataset/productos.csv.gz', delimiter='|', compression='gzip', low_memory=False)

# La ultima linea del achivo es solamente la fecha de actualizacion, por eso la borramos
comercio = comercio.iloc[:-1]
sucursales = sucursales.iloc[:-1]
productos = productos.iloc[:-1]


In [ ]:
print(comercio.shape)
print(comercio.info())
print(comercio.head(25))

In [ ]:
print(sucursales.shape)
print(sucursales.info())

In [ ]:
print(sucursales.head(25))

In [ ]:
print(sucursales['id_comercio'].unique())
print(sucursales['id_bandera'].unique())

Todas las sucursales perteneces al mismo comercio (el 10) pero a tres "banderas" diferentes, lo que es un poco curioso. De todas maneras, podemos concluir que 'id_comercio' no aporta ningún valor.

In [ ]:
print(sucursales['id_sucursal'].unique().shape)

Verificamos que las sucursales son 563 y tienen esa misma cantidad de ids únicos: con esto descartamos algún problema ahí.

In [ ]:
print(productos.shape)
print(productos.info())

In [ ]:
print(productos.head(25))

In [ ]:
print(productos['id_comercio'].unique())
print(productos['id_bandera'].unique())

Respecto al comercio y la "bandera" verificamos los mismo que ya habíamos visto con el dataset sucursales.

In [ ]:
print(productos['id_sucursal'].unique().shape)

También verificamos que al igual que el dataset sucursales tiene 563 ids de sucursal únicos. Falta verificar que en el dataset productos no haya ids de sucursales que no estén en el dataset sucursales.

In [ ]:
# Cantidad de id de producto únicos
print(productos['id_producto'].unique().shape)

In [ ]:
#Visualizamos las columnas de cada Dataframe
print("Columnas de Comercio:\n",comercio.columns)
print("Columnas de Sucursales:\n",sucursales.columns)
print("Columnas de Productos:\n",productos.columns)

Unimos los 3 dataframe en uno solo

In [ ]:
comercio=comercio[['id_bandera','comercio_bandera_nombre']]
datos=pd.merge(productos,pd.merge(comercio,sucursales,on="id_bandera"),on='id_sucursal')

In [ ]:
print(datos.shape)

El total de filas del nuevo dataset es el mismo que el dataset productos: esto quiere decir que este último no tenía id de scurursal que no estuvieran en el dataset sucursales. 

Eliminamos las columnas que no son de interes de análisis

In [ ]:
datos.columns

In [ ]:
#Eliminar columnas
datos=datos.drop(columns=['id_comercio_x','id_comercio_y','id_bandera_y',
        'sucursales_lunes_horario_atencion',
        'sucursales_martes_horario_atencion',
        'sucursales_miercoles_horario_atencion',
        'sucursales_jueves_horario_atencion',
        'sucursales_viernes_horario_atencion',
        'sucursales_sabado_horario_atencion',
        'sucursales_domingo_horario_atencion'])
datos.info()


### Descripción de las columnas

El archivo contiene 2.835.684 filas con información de los productos y sucursales.

#### Itentificadores

-  `id_comercio` : Es de tipo numérica, discreta, representa el identificador único del comercio al que pertenece el producto. Es constante para nuestro caso.

-  `id_bandera`: Es de tipo categórica, nominal. Se refiere al tipo de comercio, y está relacionado con el campo  `id_bandera ` del archivo de comercio.
-  `id_sucursal`: Es de tipo numérico, discreta. Contiene el código de la sucursal específica dentro del comercio, se relaciona con el campo de mismo nommbre del archivo sucursales.
-  `id_producto`: Es de tipo numérica, discreta. Es el identificador único del producto para el comercio.
-  `productos_ean`: Es de tipo categórica, nominal. Indica si el campo id_producto contiene el código EAN (European Article Number) del producto.
-  `productos_descripcion`: Es de tipo categórica, nominal. Contiene el nombre y descripción del producto.
-  `productos_marca`: Es de tipo categórica, nominal. Contine el nombre de la marca comercial del producto.

#### Sucursal
- `id_sucursal`: Es de tipo numérico, discreta. Es el identificador de la sucursal.

- `sucursales_tipo`: Es de tipo texto, indica el tipo de establecimiento. Relacionado con la cantidad de cajas.

- `comercio_bandera_nombre`:  Categórica, nominal.'Hipermercado Carrefour', 'Express' o 'Market'

- `sucursales_calle`: Es de tipo categórica, nominal. Representa el nombre de la calle donde está ubicada la sucursal.
- `sucursales_numero`: Es de tipo categórica, ordinal. Es el número de la dirección de la sucursal.
- `sucursales_latitud`: Es de tipo categórica, nominal. Representa la coordenada de latitud de la ubicación geográfica de la sucursal.
- `sucursales_longitud`: Es de tipo categórica, nominal. Representa la coordenada de longitud de la ubicación geográfica de la sucursal.
- `sucursales_observaciones`: Es de tipo categórica, nominal. Contiene observaciones o notas adicionales sobre la sucursal.
- `sucursales_codigo_postal`: Es de tipo categórica, nominal. Representa el código postal correspondiente a la dirección de la sucursal.
- `sucursales_localidad`: Es de tipo categórica, nominal. Indica la localidad o ciudad donde se encuentra la sucursal.
- `sucursales_barrio`: Es de tipo categórica, nominal. Indica el barrio donde se encuentra ubicada la sucursal.
- `sucursales_provincia`: Es de tipo categórica, nominal. Contine un código que indica la provincia a la que pertenece la sucursal.

#### Unidades de medida y cantidades

-  `productos_cantidad_presentacion `: Es de tipo numérica, discreta. Indica la cantidad del producto contenida en la unidad.

-  `productos_unidad_medida_presentacion `: Es de categórica, nominal. Representa la unidad de medida de la presentación del producto.
-  `productos_cantidad_referencia `: Es de tipo numérica, discreta. indica la cantidad de producto usada como base para calcular el precio de referencia.
-  `productos_unidad_medida_referencia `: Es de tipo categórica, nominal. Representa la unidad de medida de la cantidad de referencia.

#### Precios 
-  `productos_precio_lista` : Es de tipo numérico, continua. Representa el precio de lista del producto, es decir, el precio regular sin promociones.

-  `productos_precio_referencia `: Es de tipo numérico, cotinua. Corresponde al precio de referencia.

#### Promociones

-  `productos_precio_unitario_promo1 `: Es de tipo numérico, indica el precio unitario del producto en la primera promoción (si existe).

-  `productos_leyenda_promo1 `: Es de tipo categórica, nominal. Contiene la leyenda o descripción asociada a la primera promoción.
-  `productos_precio_unitario_promo2 `: Es de tipo numérica, continua. Indica el precio unitario del producto en la segunda promoción (si existe).
-  `productos_leyenda_promo2 `: Es de tipo categórica, nominal. Contiene la leyenda o descripción asociada a la segunda promoción.

### Vista general del dataset  

In [ ]:
#Mostramos las primeras filas del dataset para una visualización de parcial de los datos
datos.head()

In [ ]:
#Visualizamos la primera fila del data set
datos.iloc[0]

In [ ]:
#Utilizando el método info que nos ofrese información de cantidad de filas, columnas, nombre de las columans y el tipo de datos que asignó Python a cada una. Tambien os indica la cantidad de memoria utilizada.
datos.info()

El dataset tiene:
2.835.684 filas
18 columnas
Utiliza 822MB de memoria

### Ajustes de tipos de datos

En la respuesta del método **info** observamos que Python asignón el tipo de dato **float64** a columnas que intuimos en base al nombre y su descripción que solo continen valores enteros. Vamos a comprobar si existen valores con decimales para estas columans antes de hacer la conversión a entero.

In [ ]:

# Verificamos si el campo tiene valores decimales
print("¿El campo productos_cantidad_presentacion tiene valores decimales?:",datos['productos_cantidad_presentacion'].apply(float).mod(1).ne(0).any())
print("¿El campo productos_cantidad_referencia tiene valores decimales?:",datos['productos_cantidad_referencia'].apply(float).mod(1).ne(0).any())

En base al resultado anterior y a la tipificación de los atributos categóricos en la descripción de las columnas, realizamos la conversión de tipos de datos correspondiente

In [ ]:
### Ajustando los tipos de datos

datos['id_bandera_x']=datos['id_bandera_x'].astype(int)
datos['id_sucursal']=datos['id_sucursal'].astype(int)
datos['id_sucursal']=datos['id_sucursal'].astype('category')
datos['id_producto']=datos['id_producto'].astype(int)
datos['productos_ean']=datos['productos_ean'].astype(bool)
datos['productos_cantidad_presentacion']=datos['productos_cantidad_presentacion'].astype(int)
datos['productos_cantidad_referencia']=datos['productos_cantidad_referencia'].astype(int)
datos['productos_unidad_medida_presentacion']=datos['productos_unidad_medida_presentacion'].astype('category')
datos['productos_unidad_medida_referencia']=datos['productos_unidad_medida_referencia'].astype('category')
datos['productos_marca']=datos['productos_marca'].astype('category')
datos['sucursales_nombre']=datos['sucursales_nombre'].astype('category')
datos['sucursales_tipo']=datos['sucursales_tipo'].astype('category')
datos['sucursales_barrio']=datos['sucursales_barrio'].astype('category')
datos['sucursales_codigo_postal']=datos['sucursales_codigo_postal'].astype('category')
datos['sucursales_localidad']=datos['sucursales_localidad'].astype('category')
datos['sucursales_provincia']=datos['sucursales_provincia'].astype('category')


In [ ]:
#Cofirmamos que los cambios se hayan aplicado correctamente ejecutando nuevamente el método info
datos.info()

### Variables categóricas: exploración



Las variables identificadas como categóricas son:
- id_sucursal
- productos_unidad_medida_presentacion
- productos_marca
- productos_unidad_medida_referencia
- sucursales_nombre
- sucursales_tipo
- sucursales_barrio
- sucursales_codigo_postal
- sucursales_localidad
- sucursales_provincia

Todas son nominales.

In [ ]:
datos.describe(include='category')

Listamos los valores únicos y la cantidad de cada una de las variables junto con gráficos que nos permitan visualizar mejor los valores:

id_sucursal: no hacemos ningún análisis particular de esta variable, lo que nos interesa es analizar la variable corelacionada sucursales_nombre

productos_unidad_medida_presentacion:

In [ ]:
print(datos['productos_unidad_medida_presentacion'].unique())
sns.countplot(data=datos, x='productos_unidad_medida_presentacion', color="blue")

productos_marca:

In [ ]:
print(datos['productos_marca'].unique())

Son demasiados para poder graficarlos adecuadamente, así que listamos los 25 más frecuentes:

In [ ]:
# Obtengo las 25 marcas con más productos/precios, la frecuencia y su porcentaje
productos_marca = datos['productos_marca'].value_counts().reset_index()
productos_marca.columns = ['productos_marca', 'frecuencia']
productos_marca = productos_marca.sort_values(by="frecuencia", ascending=False)
productos_marca["porcentaje"] = productos_marca["frecuencia"] / productos_marca["frecuencia"].sum() * 100
productos_marca = productos_marca.head(25)

print(productos_marca)

Como vemos, el mayor porcentaje (8.56%) es de marca sin definir. Podemos ver que luego entre las más frecuentes siguen marcas de Carrefour, genéricas, 'Bulnez' y 'La Serenísima'. A partir de la marca séptima de mayor frecuencia, los porcentajes son menores que 1.

productos_unidad_medida_referencia:

In [ ]:
print(datos['productos_unidad_medida_referencia'].unique())
sns.countplot(data=datos, x='productos_unidad_medida_referencia', color="blue")

sucursales_nombre:

In [ ]:
print(datos['sucursales_nombre'].unique())

Son demasiados para poder graficarlos adecuadamente, así que listamos los 25 más frecuentes:

In [ ]:
# Obtengo las 25 sucursales con más productos/precios, la frecuencia y su porcentaje
sucursales_nombre = datos['sucursales_nombre'].value_counts().reset_index()
sucursales_nombre.columns = ['sucursales_nombre', 'frecuencia']
sucursales_nombre = sucursales_nombre.sort_values(by="frecuencia", ascending=False)
sucursales_nombre["porcentaje"] = sucursales_nombre["frecuencia"] / sucursales_nombre["frecuencia"].sum() * 100
sucursales_nombre = sucursales_nombre.head(25)

print(sucursales_nombre)

Como podemos ver ninguna sucursal supera el 1% de frecuencia, lo que nos dice que tiene una distribución altamente dispersa.

sucursales_tipo:

In [ ]:
print(datos['sucursales_tipo'].unique())
sns.countplot(data=datos, x='sucursales_tipo', color="blue")

sucursales_barrio:

In [ ]:
print(datos['sucursales_barrio'].unique())

plt.figure(figsize=(6, 12)) 
sns.countplot(
    data=datos,
    y='sucursales_barrio',
    order=datos['sucursales_barrio'].value_counts().index  # ordena de mayor a menor frecuencia
)
plt.tight_layout()
plt.show()

sucursales_codigo_postal:

In [ ]:
print(datos['sucursales_codigo_postal'].unique())
plt.figure(figsize=(6, 36)) 
sns.countplot(
    data=datos,
    y='sucursales_codigo_postal',
    order=datos['sucursales_codigo_postal'].value_counts().index  # ordena de mayor a menor frecuencia
)
plt.tight_layout()
plt.show()

sucursales_localidad:

In [ ]:
print(datos['sucursales_localidad'].unique())
plt.figure(figsize=(6, 24)) 
sns.countplot(
    data=datos,
    y='sucursales_localidad',
    order=datos['sucursales_localidad'].value_counts().index  # ordena de mayor a menor frecuencia
)
plt.tight_layout()
plt.show()

sucursales_provincia:

In [ ]:
print(datos['sucursales_provincia'].unique())
plt.figure(figsize=(6, 6)) 
sns.countplot(
    data=datos,
    y='sucursales_provincia',
    order=datos['sucursales_provincia'].value_counts().index  # ordena de mayor a menor frecuencia
)
plt.tight_layout()
plt.show()

### Variables numéricas: estadística descriptiva (Ricardo)

Utilizamos el método **describe** que nos muetra información sobre las principales medidas estadísticas de nuestro conjunto de datos


In [ ]:
datos.describe()

#### id_sucursal

Este atributo es el identificador de sucursal, vamos contar cuantos valores únicos hay para conocer la cantidad de Sucursales

In [ ]:
#Calculamos la cantidad de Sucursales
print( "Cantidad de Sucursales: ",datos['id_sucursal'].nunique())

#### Id_producto

In [ ]:
#Calculamos la cantidad de productos
print( "Cantidad de Productos ",datos['id_producto'].nunique())

In [ ]:
df_frecuencia = datos['id_producto'].value_counts().reset_index()
print(df_frecuencia)


Vemos que el ID de producto se repirte como máximo 563 veces, que es la cantidad de sucursales, es decir que son productos que se venden en todas las sucursales. Hay mucho productos que se venden en una sola sucursal

#### productos_cantidad_presentacion

In [ ]:
print(datos['productos_cantidad_presentacion'].describe())

Todos los valores de esta columna son igual a 1. No aporta información.

#### productos_precio_lista

In [ ]:
datos['productos_precio_lista'].describe()

In [ ]:
sns.histplot(datos['productos_precio_lista'], kde=False, bins=60)
plt.title('Distribución de precios de lista')
plt.xlabel('Precio de lista')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
print(f"La varianza del precio de lista es: {datos['productos_precio_lista'].var()}")

Observamos que los precios tiene una disperción muy grande. Vamos graficar los datos del tercer cuartil.

In [ ]:
# Calculamos el percentil 75 y filtramos los datos
q75 = datos['productos_precio_lista'].quantile(0.75)
df_filtrado = datos[datos['productos_precio_lista'] <= q75]

# Calcular medidas estadísticas
media = df_filtrado['productos_precio_lista'].mean()
mediana = df_filtrado['productos_precio_lista'].median()
moda = df_filtrado['productos_precio_lista'].mode().iloc[0]  # solo la primera moda

# Crear el histograma
sns.histplot(df_filtrado['productos_precio_lista'], kde=True, bins=60)
plt.title('Distribución de precios (hasta percentil 75)')
plt.xlabel('Precio de lista')
plt.ylabel('Frecuencia')

# Agregar líneas verticales para media, mediana y moda
plt.axvline(media, color='blue', linestyle='--', label=f'Media: {media:.2f}')
plt.axvline(mediana, color='green', linestyle='--', label=f'Mediana: {mediana:.2f}')
plt.axvline(moda, color='orange', linestyle='--', label=f'Moda: {moda:.2f}')

# Mostrar leyenda
plt.legend()
plt.show()

Como se puede apreciar en el gráfico, la distribución de este atributo presenta una **asimetría positiva** 

In [ ]:
#Para toda la muetra
print(f"Skewness = {datos['productos_precio_lista'].skew()}")

In [ ]:
#para el terer cuartil
print(f"Skewness = {df_filtrado['productos_precio_lista'].skew()}")


In [ ]:
from scipy.stats import kurtosis

# Filtrá si hay valores extremos
data = df_filtrado['productos_precio_lista'].dropna()
k = kurtosis(data, fisher=True)

print(f"Curtosis:{k}")

#### id_sucursal y coordenadas geográficas

Vamos a visializar las sucursales de los comercios en el mapa de Argenina

In [ ]:
# Usaremos un archivo GeoJSON libre para generar el mapa de Argentina 

url_argentina = 'https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson'
world = gpd.read_file(url_argentina)
# Filtrar solo Argentina
argentina = world[world['name'] == 'Argentina']
#Agrupamos los datos por código de sucursal y contamos la cantidad de prouctos de cada una
productos_por_sucursal=datos.groupby('id_sucursal').agg(
    {   'id_sucursal':'first',
        'comercio_bandera_nombre':'first',
        'sucursales_tipo':'first',
        'sucursales_latitud': 'first',
        'sucursales_longitud': 'first',
        'comercio_bandera_nombre': 'first',
        'id_bandera_x':'first',
        'id_producto': 'count' 
    }
)
#Generamos la s estructuras de datos necesarias de entrada para renderizar el mapa
sucursales_id=productos_por_sucursal['id_sucursal'].to_list()
coordenadas=list(zip(productos_por_sucursal['sucursales_longitud'],productos_por_sucursal['sucursales_latitud']))
#Creamos un mapa de color en base a los valores del campo id_bandera_x (1,2,3) que determina si el comercio es un Hipermercado, Market o Express
norm = plt.Normalize(vmin=1, vmax=3)
cmap = plt.get_cmap('viridis')  #

colores = [cmap(norm(val)) for val in productos_por_sucursal['id_bandera_x']]

escalar=productos_por_sucursal['id_producto'].max()/100
sizes=[int(val/escalar) for val in productos_por_sucursal['id_producto']]


productos_por_sucursal.head()

# Convertir a GeoDataFrame
puntos = gpd.GeoDataFrame(
    {'pv': sucursales_id},
    geometry=[Point(lon, lat) for lon, lat in coordenadas],
    crs='EPSG:4326'  # Sistema de coordenadas WGS84
)
patches = [
    mpatches.Patch(color=cmap(norm(1)), label='Hipermercado'),
    mpatches.Patch(color=cmap(norm(2)), label='Market'),
    mpatches.Patch(color=cmap(norm(3)), label='Express')
]

# Generar el gráfico
ax = argentina.plot(figsize=(20, 20), color='white', edgecolor='black')
puntos.plot(ax=ax, color=colores, marker='o', markersize=sizes)
plt.title("Sucursales Carrefour en Argentina")
plt.legend(handles=patches, loc='upper right')
plt.show()

El gráfico muestra la ubicación geográfica de las sucursales diferenciados por tipo de sucursal según el color. La dimensión de los puntos indica la cantidad de productos que se venden en esa sucursal. 

### Visualizaciones entre pares de variables


Investigamos relaciónes entre variables para descubir dependencias o relaciones entre ellas. Principalmente la variable precio_lista con el resto que es el target de nuestro problema.

Calculamos, mostramos y graficamos la matriz de correlación entre las variables más relevantes para detectar relaciones:

In [ ]:
#Declaramos funciones para calcular la matriz de correlación

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    rcorr = r - ((r - 1) ** 2) / (n - 1)
    kcorr = k - ((k - 1) ** 2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1))) if n > 1 else 0

def eta_squared(cat, num):
    groups = [num[cat == c] for c in cat.unique()]
    if len(groups) < 2:
        return 0
    fvalue, _ = ss.f_oneway(*groups)
    eta = fvalue * (len(cat) - len(groups)) / (fvalue * (len(cat) - len(groups)) + len(cat) - 1)
    return eta

def mixed_correlation_matrix(df):
    num_cols = df.select_dtypes(include=np.number).columns
    cat_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns
    all_cols = list(num_cols) + list(cat_cols)
    
    corr_matrix = pd.DataFrame(np.nan, index=all_cols, columns=all_cols)
    
    corr_numeric = df[num_cols].corr()
    corr_matrix.loc[num_cols, num_cols] = corr_numeric
    
    for c1 in cat_cols:
        for c2 in cat_cols:
            if c1 == c2:
                corr_matrix.loc[c1, c2] = 1.0
            else:
                corr_matrix.loc[c1, c2] = cramers_v(df[c1], df[c2])
    
    for c1 in cat_cols:
        for c2 in num_cols:
            val = eta_squared(df[c1], df[c2])
            corr_matrix.loc[c1, c2] = val
            corr_matrix.loc[c2, c1] = val
    
    return corr_matrix

In [ ]:
# Seleccionamos las variables más relevantes
ds_corr = datos[['id_bandera_x', 'id_sucursal', 'id_producto', 'productos_cantidad_presentacion', 'productos_unidad_medida_presentacion', 'productos_marca',
                 'productos_precio_lista', 'productos_precio_referencia', 'productos_cantidad_referencia', 'productos_unidad_medida_referencia',
                 'productos_precio_unitario_promo1', 'productos_precio_unitario_promo2', 'sucursales_tipo', 'sucursales_barrio', 'sucursales_localidad', 'sucursales_provincia']]


#Calculamos y mostramos la matriz de correlación
corr_mixta = mixed_correlation_matrix(ds_corr)
print(corr_mixta)

#Graficamos
plt.figure(figsize=(10, 8))
sns.heatmap(corr_mixta, annot=True, fmt=".2f", cmap="coolwarm", center=0, vmin=0, vmax=1)
plt.title("Matriz de correlación")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Nos interesa identificar los valores cercanos a 1 que muestran una relacion directa y cercanos a -1 que muestran una relacion inversa. Hay variables fuertemente relacionadas que en realidad nos interesan: por ejemplo variables entre sí misimas, sucursal con localidad, producto y marca, etc.



Ahora mostramos gráficos de dispersión entre variables relacionadas que nos parecen interesantes de entender mejor.

En algunos casos utilizamos los ids en lugar de los nombres y descripciones porque son muchos datos que no se visualizan adecuadamente y demoran mucho. Con los ids igual podemos hacernos una idea de la relacion entre las variables.



In [ ]:
plt.figure(figsize=(16, 16))
sns.scatterplot(data=datos, x='id_producto', y='id_sucursal')
plt.title("Relación entre producto y sucursal")
plt.show()

In [ ]:
plt.figure(figsize=(16, 16))
sns.scatterplot(data=datos, x='productos_precio_lista', y='productos_precio_unitario_promo1')
plt.title("Relación entre precio y precio promo")
plt.show()

In [ ]:
plt.figure(figsize=(16, 16))
sns.scatterplot(data=datos, x='productos_precio_lista', y='id_sucursal')
plt.title("Relación entre precio y sucursal")
plt.show()

In [ ]:
plt.figure(figsize=(16, 16))
sns.scatterplot(data=datos, x='productos_precio_lista', y='productos_marca')
plt.title("Relación entre precio y marca")
plt.yticks([])
plt.show()

In [ ]:
plt.figure(figsize=(16, 2))
sns.scatterplot(data=datos, x='productos_precio_lista', y='sucursales_tipo')
plt.title("Relación entre precio y tipo de sucursal")
plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
sns.scatterplot(data=datos, x='productos_precio_lista', y='sucursales_provincia')
plt.title("Relación entre precio y provincia")
plt.show()

In [ ]:
plt.figure(figsize=(16, 16))
sns.scatterplot(data=datos, x='productos_precio_lista', y='sucursales_localidad')
plt.title("Relación entre precio y localidad")
plt.yticks([])
plt.show()

Por último agrupamos y hacemos comparaciones por categorías entre variables relacionadas que nos parecen interesantes de entender mejor.

Si bien por el tipo de dateset estos gráficos no aportan todo el valor que sí tienen para otros tipos de datasets, visualizarnos nos ayudan a enter un poco mejor algunas rlaciones.

In [ ]:
plt.figure(figsize=(8, 8))
(datos.groupby(['id_sucursal'], observed=True)['productos_precio_lista'].mean().sort_values().plot(kind='barh', colormap='Pastel2' , title='Precios por sucursal'))
plt.yticks([])
plt.show()

Vemos una distribución curiosa de precios si agrupamos por sucursal: necesitamos entender mejor esta relación.

In [ ]:
plt.figure(figsize=(16, 16))
(datos.groupby(['productos_marca'], observed=True)['productos_precio_lista'].mean().sort_values().plot(kind='barh', colormap='Pastel2' , title='Precios por marca'))
plt.yticks([])
plt.show()

In [ ]:
plt.figure(figsize=(4, 4))
(datos.groupby(['sucursales_tipo'], observed=True)['productos_precio_lista'].mean().sort_values().plot(kind='barh', colormap='Pastel2' , title='Precios por tipo de sucursal'))
plt.show()

In [ ]:
plt.figure(figsize=(4, 4))
(datos.groupby(['sucursales_provincia'], observed=True)['productos_precio_lista'].mean().sort_values().plot(kind='barh', colormap='Pastel2' , title='Precios por provincia'))
plt.show()

In [ ]:
plt.figure(figsize=(4, 4))
(datos.groupby(['sucursales_localidad'], observed=True)['productos_precio_lista'].mean().sort_values().plot(kind='barh', colormap='Pastel2' , title='Precios por localidad'))
plt.yticks([])
plt.show()

## Revisión de valores faltantes

### Identificación de datos faltantes


In [ ]:
datos.isna().mean().round(4)*100

In [ ]:
msno.matrix(datos, figsize=(20,8))


In [ ]:
msno.heatmap(datos, fontsize=15, figsize=(15,8))
plt.title("Correlación de datos nulos", fontsize=15)
plt.show()


In [ ]:
msno.dendrogram(datos, figsize=(15,8))
plt.title("Dendograma de faltantes", fontsize=20)
plt.show()



### Analisis de faltantes

Faltantes:

- Porcentaje de faltantes en productos_precio_unitario_promo1: 95.55%
- Porcentaje de faltantes en productos_leyenda_promo1: 95.55%
- Porcentaje de faltantes en productos_precio_unitario_promo2: 100.00%
- Porcentaje de faltantes en productos_leyenda_promo2: 100.00%
- Porcentaje de faltantes en sucursales_numero: 1.78%
- Porcentaje de faltantes en sucursales_observaciones: 92.85%
- Porcentaje de faltantes en sucursales_barrio 69.74%


#### *_promo2

Veamos que pasa con `productos_precio_unitario_promo2` y `productos_leyenda_promo2`. 
En el PDF que adjunta este dataset lo describe a la promosion de tipo 2 como, una promosión especial; mientras que a la promoción 1 (`productos_precio_unitario_promo1` y `productos_leyenda_promo1`) la llama general. La clave para poder distingir la promoción de tipo y y la promición de tipo 2 es que:
- La promoción de tipo 1 no contempla segmentación de consumidor (Jubilado, Estudiante, etc), mientras que la de tipo 2 si.
- La promoción de tipo 1 no comtempla medios de pago o tarjeta de fidelizacion, mientras que la de tipo 2 si.

Podemos concluir que (MNAR) la falta del dato depende de del dato en sí mismo, no es tan descabellado asumir que no se registraron promosiones de este tipo para los productos.

#### *_promo1

`productos_precio_unitario_promo1` y `productos_leyenda_promo1` estan relacionadas, lo podemos ver en la matriz de corelacion. Los valores existentes responden a promos generales.

In [ ]:
# Creamos una función para calcular el porcentaje de nulos en una columna, agrupando por una o más columnas
def nulos_por_grupo(df, grupo_cols, target_col):
    agrupar_nulos = (
        df.groupby(grupo_cols, observed=True)[target_col]
        .apply(lambda x: x.isnull().mean() * 100)
        .reset_index(name=f'porcentaje_nulos_{target_col}')
    )
    return agrupar_nulos

In [ ]:
print(datos[~datos['productos_leyenda_promo1'].isna()][['productos_descripcion', 'productos_leyenda_promo1', 'productos_marca']])


In [ ]:
# Veamos que pasa con `sucursales_numero` y `sucursales_observaciones`

# Obtenemos las filas que tienen algún problema.
problematic_rows = datos[datos['sucursales_numero'].isna()]

# Como tenemos los datos de-normalizados por producto, removemos los duplicados para poder quedarnos solo con las sucursales que tienen problemas
problematic_rows.drop_duplicates(subset=['sucursales_latitud', 'sucursales_longitud'], keep='first', inplace=False)[['sucursales_numero', 'sucursales_calle', 'sucursales_observaciones', 'sucursales_localidad']]


#### sucursales_numero

Hay alguna correlacion entre estos datos, podemos observar en la matriz de correlacion que tiene un valor numerico de relacion positivo, pero bajo.
- Para la fila 2, podemos observar que la interesección de la calle está en `sucursales_observaciones`.
- Para la fila 3, podemos observar que el kilometro en el que se encuentra la sucursal está en `sucursales_observaciones`.
- Para la fila 33, podemos observar que en `sucrusales_calle` se encuentra la intersección en la que está la sucursal.
- Para la fila 187, podemos observar que la interesección de la calle está en `sucursales_observaciones`.

No hay una cantidad de datos faltantes significativa, aún asi, tal vez deberíamos usar directamente la geolocalización.

In [ ]:
# sucursales_barrio
deduped = datos.drop_duplicates(subset=['sucursales_latitud', 'sucursales_longitud'], keep='first', inplace=False)

nulos_barrio_por_localidad = nulos_por_grupo(deduped, 'sucursales_localidad', 'sucursales_barrio')

print("Nulos en 'barrio' por localidad:\n", nulos_barrio_por_localidad, "\n")


In [ ]:

ordenados = nulos_barrio_por_localidad.sort_values(by='porcentaje_nulos_sucursales_barrio', ascending=False)

lista_orden = nulos_barrio_por_localidad.sort_values(by='porcentaje_nulos_sucursales_barrio', ascending=False)['sucursales_localidad'].tolist()

plt.figure(figsize=(10, 16)) 
sns.barplot(
    x='porcentaje_nulos_sucursales_barrio',
    y='sucursales_localidad',
    data=nulos_barrio_por_localidad,
    order=lista_orden,
    palette='viridis' 
)

plt.title('Porcentaje de Nulos en Sucursales por Localidad')
plt.xlabel('Porcentaje de Nulos')
plt.ylabel('Localidad')
plt.xlim(0, 100) 
plt.grid(axis='x', linestyle='--', alpha=0.7) 

# Adjust layout to prevent labels from overlapping
plt.tight_layout()
plt.show()

Una hipotesis puede ser que solo las ciudades con muchas sucursales tienen el barrio asignado, pero como vemos el caso de Rosario que tiene 5 sucursales, no estan disponibles. Por lo tanto podemos concluir que es MNAR: la falta del dato depende de del dato en sí mismo.

## Detección de outliers (Joselo) 

### Exportación de dataset analizado (ADAPTAR)

In [ ]:
# MODIFICAR EL CODIGO PARA NUJESTR DATASET

import json

# Detectar columnas categóricas
categorical_cols = [col for col, dtype in carreras_24.dtypes.items() if str(dtype) == 'category']

# Guardar los tipos como strings
dtypes_str = {col: str(dtype) for col, dtype in carreras_24.dtypes.items()}

# Guardar todo en un solo JSON
info = {
    "dtypes": dtypes_str,
    "categoricals": categorical_cols
}

with open("../datasets/carreras_24_1_dtypes.json", "w") as f:
    json.dump(info, f)

# exportar CSV
carreras_24.to_csv('../datasets/carreras_24_1.csv', index=False)

---
## Conclusiones